In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging


logging.langsmith("Runnable")

LangSmith 추적을 시작합니다.
[프로젝트명]
Runnable


# RunnablePassthrough

In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [4]:
prompt = PromptTemplate.from_template("{num} 의 10배는?")
llm = ChatOpenAI(model='gpt-4o-mini')

chain = prompt | llm

In [5]:
chain.invoke({"num":5})

AIMessage(content='5의 10배는 50입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 14, 'total_tokens': 24, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-b46cff13-93cd-41cc-a8d3-2832f3fc2653-0', usage_metadata={'input_tokens': 14, 'output_tokens': 10, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [6]:
chain.invoke(5)

AIMessage(content='5의 10배는 50입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 14, 'total_tokens': 24, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-f1f7e9b3-ab43-4bae-a762-f338589fb3d3-0', usage_metadata={'input_tokens': 14, 'output_tokens': 10, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [7]:
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke({"num":10})

{'num': 10}

In [8]:
runnable_chain = {"num":RunnablePassthrough()} | prompt | llm

runnable_chain.invoke(10)

AIMessage(content='10의 10배는 100입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 14, 'total_tokens': 24, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-04e61d76-7737-4fd4-83c8-9def33936191-0', usage_metadata={'input_tokens': 14, 'output_tokens': 10, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [9]:
(RunnablePassthrough.assign(new_num=lambda x:x["num"]*3)).invoke({"num":1})

{'num': 1, 'new_num': 3}

## RunnableParallel

In [15]:
from langchain_core.runnables import RunnableParallel

# RunnableParallel 인스턴스 생성. 이 인스턴스는 여러 Runnable 인스턴스를 병렬로 실행할 수 있다.

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mulit=lambda x: x['num']*3),
    modified=lambda x: x["num"] +1,
)

In [16]:
runnable.invoke({"num":1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mulit': 3}, 'modified': 2}

In [17]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="Lama3.2-korean:latest", max_token = 500)

In [18]:
chain1 = (
    {"tech":RunnablePassthrough()}
    | PromptTemplate.from_template("1줄로 {tech} 의 정의는?")
    | llm
)
chain2 = (
    {"tech":RunnablePassthrough()}
    | PromptTemplate.from_template("1줄로 {tech} 의 용도는?")
    | llm
)

In [19]:
combined_chain = RunnableParallel(tech=chain1,use=chain2)
combined_chain.invoke("나무")

{'tech': AIMessage(content='나무는 생물체로, 주로 식물의 한 종류로서, 주로 그들의 목재와 유자를 이용하여 인간의 생활과 산업 활동에 중요한 역할을 한다.', additional_kwargs={}, response_metadata={'model': 'Lama3.2-korean:latest', 'created_at': '2024-10-24T09:39:42.88821Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 19781809125, 'load_duration': 9311336625, 'prompt_eval_count': 75, 'prompt_eval_duration': 1868455000, 'eval_count': 44, 'eval_duration': 8587433000}, id='run-96998f2f-ded8-4528-b99e-3535bd2a4120-0'),
 'use': AIMessage(content='나무는 건물, 도구, 화가의 재료, 연기 및 에너지 생산, 음식, 의류, 수단 등 다양한 목적을 위해 사용됩니다.', additional_kwargs={}, response_metadata={'model': 'Lama3.2-korean:latest', 'created_at': '2024-10-24T09:39:42.63004Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 19523637708, 'load_duration': 9312217708, 'prompt_eval_count': 75, 'prompt_eval_duration': 1870222000, 'eval_count': 41, 'eval_duration': 8326442000}, id='run

## Runnable Lambda
- 사용자 정의 함수를 맵핑할 수 있다.

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from datetime import datetime

In [39]:
def get_today(a):
    print(f"입력 받은 변수 a의 값 : {a}")
    print(f"입력받은 n의 값 : {a['n']}")
    return datetime.today().strftime("%b-%d")

In [42]:
# get_today(None)

In [7]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter
from langchain_teddynote import logging
from langchain_core.prompts import ChatPromptTemplate

logging.langsmith("Runnable")

LangSmith 추적을 시작합니다.
[프로젝트명]
Runnable


In [51]:
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 유명인 {n} 명을 나열하세요. 생년월일을 표기해주세요."
)
llm = ChatOllama(model="Lama3.2-korean:latest", max_token = 1024, temperature=0)

chain = (
    {"today":RunnableLambda(get_today),"n":itemgetter("n")}
    | prompt
    | llm 
    | StrOutputParser()
)

In [45]:
print(chain.invoke({"n":3}))

입력 받은 변수 a의 값 : {'n': 3}
입력받은 n의 값 : 3
생일이 Oct-24일인 유명인 3 명은 다음과 같습니다:

1. **Alfred Hitchcock** - 1899년 8월 13일
2. **John Lennon** - 1940년 10월 9일
3. **Michael Jackson** - 1958년 8월 29일


In [2]:
def length_function(text):
    return len(text)

In [3]:
def _multiple_length_funct(text1,text2):
    return len(text1) * len(text2)

In [9]:
def multiple_length_function(_dict):
    return _multiple_length_funct(_dict["text1"],_dict["text2"])

In [10]:
prompt = ChatPromptTemplate.from_template("{a} + {b} 는 무엇인가요?")

In [12]:
llm = ChatOllama(model="Lama3.2-korean:latest", max_token = 1024, temperature=0)
chain = ({
    "a":itemgetter("world1") | RunnableLambda(length_function),
    "b":{"text1":itemgetter("world1"),"text2":itemgetter("world2")}
    | RunnableLambda(multiple_length_function)
}
| prompt
| llm
)

/var/folders/dn/kpsvhr056zz6gvcnmgrkklzr0000gn/T/ipykernel_41106/2685144767.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="Lama3.2-korean:latest", max_token = 1024, temperature=0)


In [13]:
chain.invoke({"world1":"hello","world2":"world"})

AIMessage(content='5 + 25는 30입니다.', additional_kwargs={}, response_metadata={'model': 'Lama3.2-korean:latest', 'created_at': '2024-10-24T10:14:52.567277Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 12540696250, 'load_duration': 10583359833, 'prompt_eval_count': 74, 'prompt_eval_duration': 840969000, 'eval_count': 10, 'eval_duration': 1096873000}, id='run-f3d82b82-a26a-4f04-b970-c3b79c6fa89c-0')